In [3]:
spark.catalog.listDatabases()

[Database(name='default', description='Default Hive database', locationUri='hdfs://namenode:8020/user/hive/warehouse'),
 Database(name='kodata', description='', locationUri='hdfs://namenode:8020/user/hive/warehouse/kodata.db')]

In [3]:
spark.catalog.listTables()

[Table(name='hist_painel_covidbr', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='teste1', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='teste2', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='teste3', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='teste4', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='teste5', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='teste6', database='default', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='teste7', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [6]:
spark.read.table("kodata.t_hist_covid").show(5)

+------+------+-----+------+--------------+---------------+----+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+---------------------+--------------------+
|regiao|estado|coduf|codmun|codregiaosaude|nomeregiaosaude|data|semanaepi|populacaotcu2019|casosacumulado|casosnovos|obitosacumulado|obitosnovos|recuperadosnovos|emacompanhamentonovos|interiormetropolitana|           municipio|
+------+------+-----+------+--------------+---------------+----+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+---------------------+--------------------+
|regiao|estado| null|  null|          null|           null|null|     null|            null|          null|      null|           null|       null|            null|                 null|                 null|__HIVE_DEFAULT_PA...|
|Brasil|      | null|    76|          null|           null|null|     null|              

In [4]:
!hdfs dfs -ls /user/erick/data/input/

Found 4 items
-rw-r--r--   3 root supergroup   62492959 2021-11-28 02:42 /user/erick/data/input/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup   76520681 2021-11-28 02:42 /user/erick/data/input/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rw-r--r--   3 root supergroup   91120916 2021-11-28 02:42 /user/erick/data/input/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup    3046774 2021-11-28 02:42 /user/erick/data/input/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv


In [5]:
view_covidbr = spark.read.csv("/user/erick/data/input/", sep=";", header="true")

In [45]:
view_covidbr.show(5)

+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|interior/metropolitana|
+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|Brasil|  null|     null|   76|  null|          null|           null|2020-02-25|        9|       210147125|             0|         0|              0|          0|            null|                 null|                  null|
|Brasil|  null|     null|   76|  null|          null|           null|2020-02-26|        9|       2101471

In [7]:
from pyspark.sql.functions import *

In [61]:
view_covidbr.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- coduf: string (nullable = true)
 |-- codmun: string (nullable = true)
 |-- codRegiaoSaude: string (nullable = true)
 |-- nomeRegiaoSaude: string (nullable = true)
 |-- data: string (nullable = true)
 |-- semanaEpi: string (nullable = true)
 |-- populacaoTCU2019: string (nullable = true)
 |-- casosAcumulado: string (nullable = true)
 |-- casosNovos: string (nullable = true)
 |-- obitosAcumulado: string (nullable = true)
 |-- obitosNovos: string (nullable = true)
 |-- Recuperadosnovos: string (nullable = true)
 |-- emAcompanhamentoNovos: string (nullable = true)
 |-- interior/metropolitana: string (nullable = true)



In [46]:
view_covidbr.count()

2624943

In [86]:
spark.read.table("kodata.t_hist_covid").count()

2624947

In [44]:
view_covidbr.agg(sum("Recuperadosnovos").cast("int").alias("Recuperados"),sum("emAcompanhamentoNovos").cast("int").alias("EmAcompanhamento"),sum("casosAcumulado").cast("int").alias("casosAcumulado"),sum("casosNovos").cast("int").alias("casosNovos"),sum("obitosAcumulado").cast("int").alias("obitosAcumulado"),sum("casosNovos").cast("int").alias("casosNovos")).show()

+-----------+----------------+--------------+----------+---------------+----------+
|Recuperados|EmAcompanhamento|casosAcumulado|casosNovos|obitosAcumulado|casosNovos|
+-----------+----------------+--------------+----------+---------------+----------+
| 2147483647|       320793426|    2147483647|  56565045|      274784085|  56565045|
+-----------+----------------+--------------+----------+---------------+----------+



In [40]:
casos_reuperados = view_covidbr.agg(sum("Recuperadosnovos").cast("int").alias("Recuperados"),sum("emAcompanhamentoNovos").cast("int").alias("EmAcompanhamento"))

In [41]:
casos_reuperados.write.saveAsTable("view_covid_casos_reuperados")

In [42]:
casos_confirmados = view_covidbr.agg(sum("casosAcumulado").cast("int").alias("casosAcumulado"),sum("casosNovos").cast("int").alias("casosNovos"))

In [43]:
casos_confirmados.write.parquet("view_covid_casos_confirmados")